In [2]:
import numpy as np

### 概率求解：给定观测序列和模型参数计算观测序列出现的概率
1. 前向算法
    - 初值
    $$\alpha(i)=pi_ib_i(o_1), i=1,2,3,...N$$
    - 递推 对t=1, 2, 3, ..., T - 1
    $$\alpha_{t+1} = \left[\sum_{j=1}^N \alpha_t(j)a_{ji} \right]b_i(o_{t+1}), i=1,2,...,N$$
    - 终止
    $$P(O|\lambda) = \sum_{i=1}^N \alpha_T(i)$$

In [34]:
# 例 10.2（p200页）
# 状态集合Q = {1, 2, 3},观测集合V={红，白} 0 1
A = np.array([[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]])
B = np.array([[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]])
Pi = np.array([0.2, 0.4, 0.4])
V = ['红', '白']    # 观测值集合
# 设T = 3，观测值O = {红 白 红}
T = 3   # 时间
N = 3   # 隐藏状态数量
O = ['红', '白', '红', '红']   # 观测值
# 1. 计算初值
alpha1 = Pi * B[:, 0]

# 2. 递推计算
alpha2 = np.array([(alpha1 * A[:, i]).sum() * B[i, 1] for i in range(N)])
alpha3 = np.array([(alpha2 * A[:, i]).sum() * B[i, 0] for i in range(N)])
alpha4 = np.array([(alpha3 * A[:, i]).sum() * B[i, 0] for i in range(N)])

# 3. 终止
result = alpha4.sum()
print(alpha1)
print(alpha2)
print(alpha3)
print(result)

[0.1  0.16 0.28]
[0.077  0.1104 0.0606]
[0.04187  0.035512 0.052836]
0.0701272


In [40]:
# 封装函数
def forward(obs, V, Pi, A, B):
    N = len(Pi)
    # 计算初值
    alpha1 = Pi * B[:, 0]
    # 递推计算
    def cal_alpha(alpha, o):
        return np.array([(alpha * A[:, i]).sum() * B[i, o] for i in range(N)])
    
    for j in obs[1:]:
        o = obs.index(j)
        alpha = alpha1
        alpha1 =  cal_alpha(alpha, o)
    # 返回终值
    return alpha1.sum()

forward(O, V, Pi, A, B)

0.0701272

2. 后向算法

In [18]:
a = ['a', 'b']
a.index('a')

0